In [1]:
!pip install redis
!pip install pymysql

In [2]:
import redis, json, time, pymysql
DB_Redis = redis.Redis(host='localhost', port=6379, db=0)
DB_MYSQL = pymysql.connect("localhost","tdr","tdr", "tdr")

!mysql -utdr -ptdr tdr < db.sql

def empty_mysql(table="films"):
    cur = DB_MYSQL.cursor()
    cur.execute('DELETE FROM {} WHERE 1=1'.format(table))
    DB_MYSQL.commit()
    cur.close()

def empty_redis(table = ""):
    if table.strip() == "":
        return DB_Redis.flushall()
    else:
        return DB_Redis.delete(*DB_Redis.keys("{}:*".format(table)))

cur = DB_MYSQL.cursor()
cur.execute("INSERT INTO films (title,year,genre_list,cast_list) VALUES('foo', 2019, 'bar', 'baz')")
DB_MYSQL.commit()
cur.close()
cur = DB_MYSQL.cursor()
cur.execute('SELECT title,year,genre_list,cast_list FROM films')
print(cur.fetchall())
cur.close()
empty_mysql()
print(DB_Redis.set('foo','bar'))
print(DB_Redis.get('foo'))
print(DB_Redis.delete('foo'))
print(DB_Redis.get('foo'))

mysql: [Warning] Using a password on the command line interface can be insecure.
(('foo', 2019, 'bar', 'baz'),)
True
b'bar'
1
None


In [3]:
with open('movies.json') as f:
    datas_temp = json.load(f)

#Suppression des doublons
datas_json = []
keys = []
for film in datas_temp:
    if film['title'] not in keys:
        keys.append(film['title'])
        datas_json.append(film)
print(datas_json[0])

{'title': 'After Dark in Central Park', 'year': 1900, 'cast': [], 'genres': []}


In [4]:
empty_redis()
print("Nombre de clés Redis après nettoyage : {} ".format(len(DB_Redis.keys('*'))))
t = time.time()
for film in datas_json:
    DB_Redis.set('films:{}'.format(film['title']), json.dumps(film))
t = time.time() - t
print('Insertions de {} films dans Redis : {:0.04f} secondes'.format(len(datas_json), t))

print('{} films dans Redis'.format(len(DB_Redis.keys('films:*'))))
#avatar = json.loads(DB_Redis.get('Avatar'))
#print(avatar['year'])

Nombre de clés Redis après nettoyage : 0 
Insertions de 26791 films dans Redis : 1.4719 secondes
26791 films dans Redis


In [5]:
def mysql_insert_update(sql,values=None):
    try:
        # create a new cursor
        cur = DB_MYSQL.cursor()
        if values is None:
            cur.execute(sql)
        elif isinstance(values, list):      
            # execute the INSERT statement
            cur.executemany(sql,values)
        else:
            cur.execute(sql,value)
        # commit the changes to the database
        DB_MYSQL.commit()
        # close communication with the database
        cur.close()
    except (Exception) as error:
        print(error)

In [6]:
empty_mysql()
datas_tuple = []
for film in datas_json:
    datas_tuple.append((film['title'],str(film['year']),json.dumps(film['genres']),json.dumps(film['cast'])))
print(datas_tuple[0])
t = time.time()
mysql_insert_update("INSERT INTO films(title,year,genre_list,cast_list) VALUES(%s,%s,%s,%s)", datas_tuple)
t = time.time() - t
print('Insertions de {} films dans MySQL : {:0.04f} secondes'.format(len(datas_json), t))

cur = DB_MYSQL.cursor()
cur.execute('SELECT title,year,genre_list,cast_list FROM films')
print('{} films dans MySQL'.format(cur.rowcount))
cur.close()

('After Dark in Central Park', '1900', '[]', '[]')
Insertions de 26791 films dans MySQL : 0.5843 secondes
26791 films dans MySQL


In [7]:
#Nombre de films commençant par A
t = time.time()
res = list(map(lambda film : json.loads(film), DB_Redis.mget(DB_Redis.keys("films:[Aa]*"))))
count = len(res)
t = time.time() - t
print("{} films commençant par A ou a dans Redis en {:0.04f}s".format(count, t))
    
cur = DB_MYSQL.cursor()
t = time.time()
cur.execute("SELECT * FROM films WHERE title LIKE 'A%'")
t = time.time() - t
print("{} films commençant par A ou a dans MySQL en {:0.04f}s".format(cur.rowcount, t))
cur.close()

t = time.time()
res = list(map(lambda film : json.loads(film), DB_Redis.mget(DB_Redis.keys("films:Harry Potter*"))))
count = len(res)
t = time.time() - t
print("{} films avec Harry Potter dans Redis en {:0.04f}s".format(count, t))

cur = DB_MYSQL.cursor()
t = time.time()
cur.execute("SELECT * FROM films WHERE title LIKE 'Harry Potter%'")
t = time.time() - t
print("{} films avec Harry Potter dans MySQL en {:0.04f}s".format(cur.rowcount, t))
cur.close()

1578 films commençant par A ou a dans Redis en 0.0435s
1578 films commençant par A ou a dans MySQL en 0.0413s
8 films avec Harry Potter dans Redis en 0.0058s
8 films avec Harry Potter dans MySQL en 0.0147s
